# Modelos Sequence-to-Sequence

Hasta el momento hemos trabajado con dos tipos de procesamiento de secuencia: el caso donde a cada elemento de la secuencia se le asigna una etiqueta y el caso donde a una secuencia se le asigna un valor, ya sea categórico o contínuo. Estos son casos útiles y comunes. Sin embargo, falta un caso general. Y este es encontrar una secuencia a partir de otra secuencia, las dos de tamaño arbitrario. A estos modelos se les llama sequence-to-sequence o seq2seq. 

En este ejercicio vamos a implementar dos variantes del mismo. Para ejemplificar su uso, ¡vamos a usar un dataset multilingue! Le enseñaremos a nuestro modelo a conjugar palabras. En el ejemplo lo haremos en tres idiomas, pero el dataset original contiene 100 idiomas con los cuáles se puede probar. Para mas información puede consultar el Sharedtask Sigmorphon en la sigueinte dirección:

https://github.com/sigmorphon/conll2018



## Preparar los datos

Tomaremos el ejemplo de español primero. Dado que ya entendemos el concepto de preparar datos de ocaciones anteriores pondemos todo el proceso en una clase, que puede ser reusada de manera genérica para diferentes idiomas. Sin embargo, los datos esta vez son diferentes a lo que ya habíamos visto.

In [1]:
import csv
import pandas as pd
import numpy as np
import io
import os
import time

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense
from tensorflow.keras.utils import to_categorical, get_file
import tensorflow as tf

Descargamos los conjuntos de datos que vamos a usar. En este caso están accesibles libremente. 

In [ ]:
# Download the file
path_to_train_low = get_file(
    'spanish-train-low', origin='https://raw.githubusercontent.com/sigmorphon/conll2018/master/task1/all/spanish-train-low')
path_to_train_medium = get_file(
    'spanish-train-medium', origin='https://raw.githubusercontent.com/sigmorphon/conll2018/master/task1/all/spanish-train-medium')
path_to_dev = get_file(
    'spanish-dev', origin='https://raw.githubusercontent.com/sigmorphon/conll2018/master/task1/all/spanish-dev')
path_to_test = get_file(
    'spanish-test', origin='https://raw.githubusercontent.com/sigmorphon/conll2018/master/task1/all/spanish-test')



Nuevamente preparamos los datos para que puedan ser ingresados a la red neuronal.

In [3]:
input_to_index = {"PAD":0,"EOS":1, "BOS":2, "OOV":3}
output_to_index = {"PAD":0,"EOS":1, "BOS":2,"OOV":3}


def load_data(path):
    data = io.open(path, encoding='UTF-8').read().strip().splitlines()
    symbols = list()
    input_chars = list()
    output_chars = list()
    final_data = list()
    for line in data:
        line =  line.split('\t')
        line[2] = line[2].split(";")
        line[1] = list(line[1])
        line[0] = list(line[0])
        tags = list()
        for tag in line[2]:
            tags.append("<"+tag+">")
        
        input_string = "<BOS> " + " ".join(tags) +" " + " ".join(line[0]) + " <EOS>"
        output_string = "<BOS> " + " ".join(line[1]) + " <EOS>"
        
        final_data.append((input_string, output_string))
    
    return zip(*final_data)

train_input_med, train_output_med = load_data(path_to_train_medium)
train_input_low, train_output_low  = load_data(path_to_train_low)
dev_input, dev_output = load_data(path_to_dev)
test_input, test_output = load_data(path_to_test)

print(train_input_med[0])
print(train_output_med[0])


<BOS> <V> <NEG> <IMP> <3> <PL> r e i t e r a r <EOS>
<BOS> n o   r e i t e r e n <EOS>


Para este problema en particular agregmos una serie de tags que an a caracterizar la información morfológica con la cual queremos conjugar nuestra raíz verbal. Es de notar que también usamos los tags de Begin of sentece y end of sentence. Para el modelo seq2seq esto es importante, dado que va a iniciar a generar salida a partir de un BOS.

Ahora vamos a extraer nuevamente nuestro vocabulario para poder darle un índice a cada tag o caracter. 

In [4]:
def get_index(data):
    word_to_index = {"<PAD>":0, "<OOV>":1}
    index_to_word = {0:"<PAD>", 1:"<OOV>"}

    voc = list()
    for line in data:
        voc += line.split()
    voc = list(set(voc))
    for word in voc:
        word_to_index[word] = len(word_to_index)
        index_to_word[len(index_to_word)] = word

    return word_to_index, index_to_word

input_to_index, index_to_input = get_index(train_input_med)
output_to_index, index_to_output = get_index(train_output_med)

print(input_to_index)
print(output_to_index)

{'<PAD>': 0, '<OOV>': 1, 'o': 2, 'h': 3, '<3>': 4, '<2>': 5, '<1>': 6, '<POS>': 7, 'f': 8, '<PL>': 9, 'l': 10, 'ü': 11, '<PST>': 12, 'c': 13, '<COND>': 14, 'd': 15, 'v': 16, '<NEG>': 17, 'n': 18, 'ñ': 19, '<SG>': 20, 'j': 21, '<V.CVB>': 22, '<SBJV>': 23, '<V>': 24, '<MASC>': 25, '<NFIN>': 26, '<BOS>': 27, '<IMP>': 28, 'i': 29, 'í': 30, '<EOS>': 31, 'b': 32, '<PRS>': 33, 'm': 34, '<IPFV>': 35, 'z': 36, 'y': 37, 's': 38, '<V.PTCP>': 39, '<FEM>': 40, 'e': 41, 'u': 42, '<FUT>': 43, '<IND>': 44, 'a': 45, 'x': 46, 'p': 47, 't': 48, 'r': 49, '<LGSPEC1>': 50, 'g': 51, '<PFV>': 52, 'q': 53}
{'<PAD>': 0, '<OOV>': 1, 'o': 2, 'h': 3, 'ó': 4, 'f': 5, 'l': 6, 'ü': 7, 'c': 8, 'd': 9, 'v': 10, 'n': 11, 'ñ': 12, 'j': 13, '<BOS>': 14, 'i': 15, 'í': 16, '<EOS>': 17, 'b': 18, 'm': 19, 'y': 20, 'z': 21, 'é': 22, 's': 23, 'e': 24, 'u': 25, 'á': 26, 'ú': 27, 'a': 28, 'x': 29, 'p': 30, 't': 31, 'r': 32, 'g': 33, 'q': 34}


Y convertimos todas nuestras entradas a enteros.

In [5]:

def to_ints(data, token_to_int):
    new_data = list()
    for line in data:
        new_line = list()
        for word in line.split():
            if not word in token_to_int.keys():
                new_line.append(token_to_int["<OOV>"])
                print("not seen", word)
            else:
                new_line.append(token_to_int[word])
        new_data.append(new_line)
    
    return(new_data)

train_med_X = to_ints(train_input_med, input_to_index)
train_med_Y = to_ints(train_output_med, output_to_index)

dev_X = to_ints(dev_input, input_to_index)
dev_Y = to_ints(dev_output, output_to_index)

train_med_X = tf.keras.preprocessing.sequence.pad_sequences(train_med_X, padding='post')
train_med_Y = tf.keras.preprocessing.sequence.pad_sequences(train_med_Y, padding='post')

print(train_input_med[0])
print(train_med_X[0])

<BOS> <V> <NEG> <IMP> <3> <PL> r e i t e r a r <EOS>
[27 24 17 28  4  9 49 41 29 48 41 49 45 49 31  0  0  0  0  0  0  0]


Preparamos nuestros datos para ser usados por el tipo de datos de tensorflow Fataset y definimos nuestros hyper parámetros. 

In [7]:
BUFFER_SIZE = len(train_med_X)
BATCH_SIZE = 20
MAX_LENGTH = 40
steps_per_epoch = len(train_med_X)//BATCH_SIZE
embedding_dim = 100
units = 1024
vocab_inp_size = len(input_to_index)
vocab_tar_size = len(output_to_index)
SAVE_EACH = 2

dataset = tf.data.Dataset.from_tensor_slices((train_med_X, train_med_Y)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)


## El modelo encoder-decoder

Definamos el encoder. En este caso para tener mayor control sobre el comportamiento del encoder vamos usar 

In [8]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))


Ahora usamos el modelo de attención de Bahdanau, tal como hemos discutido en la parte teórica.

In [9]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):

    hidden_with_time_axis = tf.expand_dims(query, 1)
    score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

    attention_weights = tf.nn.softmax(score, axis=1)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)
    
    return context_vector, attention_weights


Ahora definiremos el decoder

In [10]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    context_vector, attention_weights = self.attention(hidden, enc_output)
    x = self.embedding(x)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    output, state = self.gru(x)
    output = tf.reshape(output, (-1, output.shape[2]))
    x = self.fc(output)

    return x, state, attention_weights

Definimos nuestra función de pérdida personalizada.

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)


Creamos los objetos de encoder y decoder.

In [12]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)


Guardamos los mejores modelos.

In [13]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)


Definimos una función de distancia que será útil en la evaluación

In [36]:
def distance(str1, str2):
    """Simple Levenshtein implementation for evalm."""
    m = np.zeros([len(str2)+1, len(str1)+1])
    for x in range(1, len(str2) + 1):
        m[x][0] = m[x-1][0] + 1
    for y in range(1, len(str1) + 1):
        m[0][y] = m[0][y-1] + 1
    for x in range(1, len(str2) + 1):
        for y in range(1, len(str1) + 1):
            if str1[y-1] == str2[x-1]:
                dg = 0
            else:
                dg = 1
            m[x][y] = min(m[x-1][y] + 1, m[x][y-1] + 1, m[x-1][y-1] + dg)
    return int(m[len(str2)][len(str1)])



Predecimos y calculamos accuracy

In [40]:
def predict(inp, targ, max_samples=10):
    
    output_data = list()
    i = 0
    
    for inputs, outputs in zip(inp, targ):
        if i >= max_samples:
            break
        i += 1
        inputs = tf.expand_dims(inputs,0)
        result = ''
        predicted_ids = list()

        hidden = [tf.zeros((1, units))]
        enc_out, enc_hidden = encoder(inputs, hidden)

        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([output_to_index['<BOS>']], 0)

        for t in range(MAX_LENGTH):
            predictions, dec_hidden, attention_weights = decoder(dec_input,dec_hidden,enc_out)
            predicted_id = tf.argmax(predictions[0]).numpy()
    
            result += index_to_output[predicted_id]

            if index_to_output[predicted_id] == '<EOS>':
                break

            dec_input = tf.expand_dims([predicted_id], 0)
            
        output_data.append((result, predicted_ids, outputs))
        #print(result)    
    correct = 0
    total = 0
    dist = 0
    for pred, pred_ids, gold_ids in output_data:
        gold = "".join([index_to_output[i] for i in gold_ids[1:]])
        if pred == gold:
            correct += 1
            print(pred, "==", gold)

        else:
            print(pred, "!=", gold)
        total += 1
        dist += distance(pred, gold) 
    
    print("Accuracy:", correct/total)
    print("Distance:", dist/total)   
    return output_data

Definimos nuestra función de entrenamiento.

In [41]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([input_to_index['<BOS>']] * BATCH_SIZE, 1)

    for t in range(1, targ.shape[1]):
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
      loss += loss_function(targ[:, t], predictions)
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  
  return batch_loss

Y entrenamos nuestro modelo!

In [ ]:
EPOCHS = 20
print("Epoch", epoch)

for epoch in range(EPOCHS):
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

  # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % SAVE_EACH == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)
    
    predict(dev_X, dev_Y, max_samples=40)

    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))

Epoch 3
compágan<EOS> != compagináramos<EOS>
encasquillaría<EOS> == encasquillaría<EOS>
precalinto<EOS> != precalenté<EOS>
aradiadas<EOS> != radiadas<EOS>
eputearemos<EOS> != putearen<EOS>
originastes<EOS> != originasteis<EOS>
iaguiaras<EOS> != marginarais<EOS>
restregarías<EOS> == restregarías<EOS>
turnarán<EOS> == turnarán<EOS>
acacon<EOS> != acacheteamos<EOS>
seguin<EOS> != siguen<EOS>
selfiara<EOS> == selfiara<EOS>
aligeremos<EOS> != aligerásemos<EOS>
tronchara<EOS> == tronchara<EOS>
encalécomos<EOS> != encalabocemos<EOS>
rentastrestuestrestuestrestuestrestuestr != rentaste<EOS>
enhebramos<EOS> == enhebramos<EOS>
adareis<EOS> != adarvares<EOS>
ose<EOS> != cairelaba<EOS>
menudease<EOS> == menudease<EOS>
mandonea<EOS> != mandoneando<EOS>
prolifera<EOS> != proliferaría<EOS>
noescoce<EOS> != noescueza<EOS>
facilitar<EOS> == facilitar<EOS>
otoseseseseseseseseseseseseseseseseseses != tosiésemos<EOS>
pendejeado<EOS> == pendejeado<EOS>
satisfocoramos<EOS> != satisficiéramos<EOS>
auspiciamo

Como hemos podido ver, los modelos sequence to sequence pueden modelar una gran variedad de problemas. Gran parte de ellos pueden aprovechar el poder de estas redes únicamente adecuando los datos de entrada para obtener los resultados deseamos. 

Para mejorar el rendimiento de nuestra red es posible mejorar su rendimiento con diferentes técnicas: multi task training o transfer learning.

Modificaciones sugeridas para el futuro:
* Implementar transfer learning usando datasets parecidos al español, cómo intaliano o portugués para poder mejorar el rendimiento de español.
